In [1]:
import os
from textwrap import wrap

In [2]:
from importlib import reload

In [117]:
reload(langseeact)
reload(langseeact.tools)
reload(langseeact.prompt)

<module 'langseeact.prompt' from '/Users/csg/git-work/SeeAct-LangChain/scratch/langseeact/prompt.py'>

In [118]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [121]:
from langseeact.tools import generate_option_name
from langseeact.browser_helper import select_option
from langseeact.format_prompt import format_choices
from langseeact.format_prompt_utils import (
    format_options,
    generate_new_query_prompt,
    generate_new_referring_prompt,
)
from langseeact.prompt import PromptMaker, generate_prompt_template

In [120]:
import logging
import textwrap
import langseeact
from langseeact.tools import Browser
import base64

In [6]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
def _setup_logger(main_path=None, redirect_to_dev_log=False):
    """Set up a logger to log to both file and console within the main_path."""
    logger_name = 'SeeActAgent'
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.INFO)
    if not logger.handlers:  # Avoid adding handlers multiple times
        # Create a file handler for writing logs to a file
        if main_path:
            log_filename = 'agent.log'
            f_handler = logging.FileHandler(os.path.join(main_path, log_filename))
            f_handler.setLevel(logging.INFO)

            file_formatter = logging.Formatter('%(asctime)s - %(message)s')

            f_handler.setFormatter(file_formatter)

            logger.addHandler(f_handler)

        # Create a console handler for printing logs to the terminal
        c_handler = logging.StreamHandler()
        c_handler.setLevel(logging.INFO)

        # Create formatters for file and console handlers
        console_formatter = logging.Formatter('%(message)s')

        # Set formatters for file and console handlers
        c_handler.setFormatter(console_formatter)

        # Add the handlers to the logger
        if not redirect_to_dev_log:  # Only add console handler if not redirecting to dev log
            logger.addHandler(c_handler)

    return logger

In [77]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o",
    stop=["\nObservation", "Observation"]
)

In [8]:
logger = _setup_logger()

In [9]:
browser = langseeact.tools.Browser(logger=logger)

In [10]:
await browser.start()

Loaded website: https://www.google.com/


In [11]:
elements = await browser.get_current_page_elements()

In [13]:
choices = format_choices(elements)
options = format_options(choices)

In [57]:
print(choice_text)

If none of these elements match your target element or your target action doesn't involve an element, please select U.
A. (0.035, 0.042) <a">About</a>
B. (0.078, 0.042) <a">Store</a>
C. (0.801, 0.042) <a">Gmail</a>
D. (0.843, 0.042) <a">Images</a>
E. (0.946, 0.042) <a">Sign in</a>
F. (0.887, 0.043) <a role="button"">aria-label="Google apps"</a>
G. (0.831, 0.235) <iframe role="presentation"">name="callout"</iframe>
H. (0.5, 0.242) <img">alt="Google"</img>
I. (0.482, 0.375) <textarea role="combobox"">aria-label="Search" name="q" title="Search"</textarea>
J. (0.44, 0.474) <input role="button" type="submit"">aria-label="Google Search" name="btnK" value="Google Search"</input>
K. (0.554, 0.474) <input role="button" type="submit"">aria-label="I'm Feeling Lucky" name="btnI" value="I'm Feeling Lucky"</input>
L. (0.5, 0.5) <html">About Store GmailImages Sign in Advertising Business How Search works Our third decade of climate action: join us Privacy Terms Settings</html>
M. (0.397, 0.965) <img"

In [15]:
choice_text = format_options(choices)

In [55]:
choice_text

'If none of these elements match your target element or your target action doesn\'t involve an element, please select U.\nA. (0.035, 0.042) <a">About</a>\nB. (0.078, 0.042) <a">Store</a>\nC. (0.801, 0.042) <a">Gmail</a>\nD. (0.843, 0.042) <a">Images</a>\nE. (0.946, 0.042) <a">Sign in</a>\nF. (0.887, 0.043) <a role="button"">aria-label="Google apps"</a>\nG. (0.831, 0.235) <iframe role="presentation"">name="callout"</iframe>\nH. (0.5, 0.242) <img">alt="Google"</img>\nI. (0.482, 0.375) <textarea role="combobox"">aria-label="Search" name="q" title="Search"</textarea>\nJ. (0.44, 0.474) <input role="button" type="submit"">aria-label="Google Search" name="btnK" value="Google Search"</input>\nK. (0.554, 0.474) <input role="button" type="submit"">aria-label="I\'m Feeling Lucky" name="btnI" value="I\'m Feeling Lucky"</input>\nL. (0.5, 0.5) <html">About Store GmailImages Sign in Advertising Business How Search works Our third decade of climate action: join us Privacy Terms Settings</html>\nM. (0.

In [47]:
prompt_maker = PromptMaker()

In [113]:
PromptMaker()._initialize_prompts()["system_prompt"]

{'system_prompt': 'You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.',
 'action_space': '\n    Here are the descriptions of all allowed actions:\n\n    No Value Operations:\n    - CLICK: Click on a webpage element using the mouse.\n    - HOVER: Move the mouse over a webpage element without clicking.\n    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n    - SCROLL UP: Scroll the webpage upwards by half of the window height.\n    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n    - PRESS HOME: Scroll to the top of the webpage.\n    - PRESS END: Scroll to the bottom of the webpage.\n    - PRESS PAGEUP: Scroll up by one window height.\n    - PRESS PAGEDOWN: Scrol

In [22]:
task = 'open a tab and go to the site "www.amazon.com"'

In [33]:
prompt_maker.generate_prompt("{task}",None,choices)[0] == prompts["system_prompt"] + "\n" + prompts["action_space"]

True

In [ ]:
# System Prompt
query_text += "You are asked to complete the following task: "

# Task Description
query_text += task
query_text += "\n\n"

# Previous Actions
previous_action_text = "Previous Actions:\n"
if previous_actions is None:
    previous_actions = []
for action_text in previous_actions:
    previous_action_text += action_text
    previous_action_text += "\n"
query_text += previous_action_text
query_text += "\n"

In [38]:
print(prompts["question_description"])

The screenshot below shows the webpage you see. Think step by step before outlining the next action step at the current stage. Clearly outline which element in the webpage users will operate with as the first next target element, its detailed location, and the corresponding operation.

    To be successful, it is important to follow the following rules: 
    1. You should only issue a valid action given the current observation. 
    2. You should only issue one action at a time
    3. For handling the select dropdown elements on the webpage, it's not necessary for you to provide completely accurate options right now. The full list of options for these elements will be supplied later.
    4. Unlike humans, for typing (e.g., in text areas, text boxes) and selecting (e.g., from dropdown menus or <select> elements), you should try directly typing the input or selecting the choice, bypassing the need for an initial click. 
    5. You should not attempt to create accounts, log in or do the f

In [60]:
def format_previous_actions_to_str(previous_actions):
    # Previous Actions
    previous_action_text = "Previous Actions:\n"
    if previous_actions is None:
        previous_actions = []
    for action_text in previous_actions:
        previous_action_text += action_text
        previous_action_text += "\n"
    return previous_action_text

In [84]:
await browser.take_screenshot()

In [ ]:
encoded_image = encode_image(browser.screenshot_path)

In [114]:
prompt_human = \
        """You are asked to complete the following task: {task}

        {previous_actions}

        The screenshot below shows the webpage you see. Think step by step before outlining the next action step at the current stage. Clearly outline which element in the webpage users will operate with as the first next target element, its detailed location, and the corresponding operation.

        To be successful, it is important to follow the following rules: 
        1. You should only issue a valid action given the current observation. 
        2. You should only issue one action at a time
        3. For handling the select dropdown elements on the webpage, it's not necessary for you to provide completely accurate options right now. The full list of options for these elements will be supplied later.
        4. Unlike humans, for typing (e.g., in text areas, text boxes) and selecting (e.g., from dropdown menus or <select> elements), you should try directly typing the input or selecting the choice, bypassing the need for an initial click. 
        5. You should not attempt to create accounts, log in or do the final submission. 
        6. Terminate when you deem the task complete or if it requires potentially harmful actions.
        7. Do not generate same action as the previous one, try different ways if keep failing
        8. When there is a floating banner like ads, login, or survey floating taking more than 30% of the page, close the floating banner to proceed, the close button could look like a x on the right top corner, or choose NO THANKS to close it.
        9. When there is a floating banner on top or bottom of the page like cookie policy taking less than 30% of the page, ignore the banner to proceed.  
        10. After typing text into search or text input area, the next action is normally PRESS ENTER
        11. When there are bouding boxes in the screenshot, interact with the elements in the bounding boxes
        12. When there are multiple clickable buttons having the same value, choose the one with less obstacles in the screenshot.        
        """

# human_message = HumanMessage(
#     content=[
#         {"type": "text", "text": prompt_human},
#         {
#             "type": "image_url",
#             "image_url": {"url": "data:image/png;base64,{encoded_image}"},
#         },
#     ],
# )

human_message = HumanMessagePromptTemplate.from_template([
        {"type": "text", "text": prompt_human},
        {
            "type": "image_url",
            "image_url": {"url": "data:image/png;base64,{encoded_image}"},
        },
    ],)


prompt_system = """You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.

                Here are the descriptions of all allowed actions:

                No Value Operations:
                - CLICK: Click on a webpage element using the mouse.
                - HOVER: Move the mouse over a webpage element without clicking.
                - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.
                - SCROLL UP: Scroll the webpage upwards by half of the window height.
                - SCROLL DOWN: Scroll the webpage downwards by half of the window height.
                - PRESS HOME: Scroll to the top of the webpage.
                - PRESS END: Scroll to the bottom of the webpage.
                - PRESS PAGEUP: Scroll up by one window height.
                - PRESS PAGEDOWN: Scroll down by one window height.
                - CLOSE TAB: Close the current tab in the browser.
                - NEW TAB: Open a new tab in the browser.
                - GO BACK: Navigate to the previous page in the browser history.
                - GO FORWARD: Navigate to the next page in the browser history.
                - TERMINATE: End the current task, typically used when the task is considered complete or requires potentially harmful actions.
                - NONE: Indicates that no action is necessary at this stage. Used to skip an action or wait.

                With Value Operations:
                - SELECT: Choose an option from a dropdown menu or <select> element. The value indicates the option to select.
                - TYPE: Enter text into a text area or text box. The value is the text to be typed.
                - GOTO: Navigate to a specific URL. The value is the URL to navigate to.
                - SAY: Output answers or other information you want to tell the user.
                - MEMORIZE: Keep some content into action history to memorize it.
                """

system_message = SystemMessagePromptTemplate.from_template([
        {"type": "text", "text": prompt_system},
    ],)

prompts = [system_message,
          human_message,]

chat_template = ChatPromptTemplate.from_messages(prompts)


In [122]:
prompt_template = generate_prompt_template()

In [124]:
llm_w_prompt = {"task": lambda x: x["task"],
                "previous_actions": lambda x: format_previous_actions_to_str(x["previous_actions"]),
                "encoded_image": lambda x: None if x["encoded_image"] is None else encode_image(x["encoded_image"])}\
                    | prompt_template | llm

In [125]:
result = llm_w_prompt.invoke({"task": "Open a new tab and navigate to 'www.amazon.com'", 
                              "previous_actions": [],
                              "encoded_image": browser.screenshot_path})

In [126]:
print(result.content)

The next action is to open a new tab and navigate to 'www.amazon.com'. 

1. **Action**: NEW TAB

After opening the new tab, the next step will be to navigate to the specified URL.


In [ ]:
print(result.content)

To complete the task of opening a new tab and navigating to 'www.amazon.com', the first step is to open a new tab in the browser.

**Action:**
- **NEW TAB**: Open a new tab in the browser.

This action will create a new tab where we can then navigate to the specified URL.


In [61]:
print(prompt_maker.generate_prompt("{task}",["{previous1}","{previous2}"],choices)[0])

You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.

    Here are the descriptions of all allowed actions:

    No Value Operations:
    - CLICK: Click on a webpage element using the mouse.
    - HOVER: Move the mouse over a webpage element without clicking.
    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.
    - SCROLL UP: Scroll the webpage upwards by half of the window height.
    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.
    - PRESS HOME: Scroll to the top of the webpage.
    - PRESS END: Scroll to the bottom of the webpage.
    - PRESS PAGEUP: Scroll up by one window height.
    - PRESS PAGEDOWN: Scroll down by one window height.
    - CLOSE TAB: Close

In [30]:
prompts = PromptMaker()._initialize_prompts()

In [124]:
print("\n".join(textwrap.wrap(PromptMaker()._initialize_prompts()["value_format"])))

VALUE: Provide additional input based on ACTION. (If it doesn't
involve a value, write "None"


In [20]:
task = "{task}"
previous_actions = ["{previous_actions1}", "{previous_actions2}"]
choices = ["{choice1}", "{choice2}"]

In [194]:
choices

In [43]:
from typing import List, Optional

In [ ]:
{"task": x["task"],
    "previous_actions": lambda x: format_log_to_str(x["previous_actions"]),}

In [44]:
def generate_chat_message(task: str, 
                          previous_actions: Optional[List[str]]=None, 
                          choices: Optional[List[str]]=None) -> ChatPromptTemplate:
    prompts = PromptMaker()._initialize_prompts()

    
    system_prompt_input = prompts["system_prompt"]
    action_space_input = prompts["action_space"]
    question_description_input = prompts["question_description"]
    referring_input = prompts["referring_description"]
    element_format_input = prompts["element_format"]
    action_format_input = prompts["action_format"]
    value_format_input = prompts["value_format"]

    prompts = generate_new_query_prompt(system_prompt=system_prompt_input + "\n" + action_space_input, 
                                        task=task, 
                                        previous_actions=previous_actions,
                                        question_description=question_description_input)
    
    prompts = [
               ("system", prompts[0]),
               ("human", prompts[1]),
            ]
    if choices is not None and len(choices)>0:
        prompts.append(("ai", generate_new_referring_prompt(referring_description=referring_input, 
                        element_format=element_format_input, 
                        action_format=action_format_input,
                        value_format=value_format_input,
                        choices=choices)))
    

    chat_template = ChatPromptTemplate.from_messages(prompts)
    return chat_template

In [45]:
generate_chat_message()

ChatPromptTemplate(input_variables=[], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.\n\n    Here are the descriptions of all allowed actions:\n\n    No Value Operations:\n    - CLICK: Click on a webpage element using the mouse.\n    - HOVER: Move the mouse over a webpage element without clicking.\n    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n    - SCROLL UP: Scroll the webpage upwards by half of the window height.\n    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n    - PRESS HOME: Scroll to the top of the webpage.\n    - PRESS END: Scroll to the bottom of the

In [191]:
task = "task"
previous_actions = ["previous_actions1", "previous_actions2"]
choices = None#["choice1", "choice2"]

In [ ]:
{}

In [193]:
generate_chat_message(task,previous_actions,choices).format_messages()

[SystemMessage(content='You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.\n\n    Here are the descriptions of all allowed actions:\n\n    No Value Operations:\n    - CLICK: Click on a webpage element using the mouse.\n    - HOVER: Move the mouse over a webpage element without clicking.\n    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n    - SCROLL UP: Scroll the webpage upwards by half of the window height.\n    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n    - PRESS HOME: Scroll to the top of the webpage.\n    - PRESS END: Scroll to the bottom of the webpage.\n    - PRESS PAGEUP: Scroll up by one window height.\n    - PRESS PAGEDOWN: Scroll down by one 

In [142]:
choices

[]

In [144]:
prompt_maker.tasks

['Find the pdf of the paper "GPT-4V(ision) is a Generalist Web Agent, if Grounded"']

In [143]:

task = prompt_maker.tasks[-1]
task = 'open a tab and go to the site "www.amazon.com"'

In [145]:
prompt = prompt_maker.generate_prompt(task, None, choices=choices)

In [146]:
prompt

['You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.\n\n    Here are the descriptions of all allowed actions:\n\n    No Value Operations:\n    - CLICK: Click on a webpage element using the mouse.\n    - HOVER: Move the mouse over a webpage element without clicking.\n    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n    - SCROLL UP: Scroll the webpage upwards by half of the window height.\n    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n    - PRESS HOME: Scroll to the top of the webpage.\n    - PRESS END: Scroll to the bottom of the webpage.\n    - PRESS PAGEUP: Scroll up by one window height.\n    - PRESS PAGEDOWN: Scroll down by one window height.\n    - 

In [109]:
await browser.take_screenshot()

In [110]:
encoded_image = encode_image(browser.screenshot_path)

In [147]:
system_message

SystemMessage(content=[{'type': 'text', 'text': 'You are assisting humans doing web navigation tasks step by step. At each stage, you can see the webpage by a screenshot and know the previous actions before the current step decided by yourself that have been executed for this task through recorded history. You need to decide on the first following action to take.\n\n    Here are the descriptions of all allowed actions:\n\n    No Value Operations:\n    - CLICK: Click on a webpage element using the mouse.\n    - HOVER: Move the mouse over a webpage element without clicking.\n    - PRESS ENTER: Press the Enter key, typically to submit a form or confirm an input.\n    - SCROLL UP: Scroll the webpage upwards by half of the window height.\n    - SCROLL DOWN: Scroll the webpage downwards by half of the window height.\n    - PRESS HOME: Scroll to the top of the webpage.\n    - PRESS END: Scroll to the bottom of the webpage.\n    - PRESS PAGEUP: Scroll up by one window height.\n    - PRESS PAGE

In [148]:

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

In [153]:
chain = chat_template | llm

In [ ]:
ch

In [155]:
chain.invoke({"name": "Chris", "user_input": "Great, can you please get me a cup of coffee?"})

AIMessage(content="I wish I could help with that! While I can't physically get you a cup of coffee, I can certainly help you with tips on making a great cup or finding a good coffee shop nearby. What do you need?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 56, 'total_tokens': 100, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f51c591-dbea-4760-a786-6b7121805845-0', usage_metadata={'input_tokens': 56, 'output_tokens': 44, 'total_tokens': 100})

In [195]:
system_message = SystemMessage(
    content=[
        {"type": "text", "text": prompt[0]},
    ],
)

human_message = HumanMessage(
    content=[
        {"type": "text", "text": prompt[1]},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{encoded_image}"},
        },
    ],
)

messages = [system_message, human_message]
output0 = llm.invoke(messages)

In [196]:
print(output0.content)

The current webpage is blank, indicating that no actions have been taken yet. The task is to open a new tab and navigate to "www.amazon.com".

The next action is to open a new tab.

**Action**: NEW TAB


In [101]:
system_message = SystemMessage(
    content=[
        {"type": "text", "text": prompt[0]},
    ],
)

human_message = HumanMessage(
    content=[
        {"type": "text", "text": prompt[1]},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{encoded_image}"},
        },
        {"type": "text", "text": prompt[2]},
    ],
)

ai_message = AIMessage(
    content=[
        {"type": "text", "text": output0.content},
    ],
) 

messages = [system_message, human_message, ai_message]
output = llm.invoke(messages)

In [102]:
print(output.content)

ELEMENT: U

ACTION: NEW TAB

VALUE: None


In [ ]:
print(result.content)

Based on the provided instructions and the task at hand, the first step is to identify the search functionality on the webpage to look for the paper "GPT-4V(ision) is a Generalist Web Agent, if Grounded". 

Assuming the webpage has a search bar, the next action would be to type the title of the paper into the search bar. If the search bar is not immediately visible, we may need to scroll or navigate to find it.

Let's proceed with the following action:

**Action**: TYPE
**Value**: GPT-4V(ision) is a Generalist Web Agent, if Grounded

This action will enter the paper's title into the search bar. If the search bar is not visible, please provide a screenshot or more context so I can adjust the action accordingly.


In [ ]:
print(result.content)

The next step is to type the search query into the search bar. 

**Action**: TYPE

**Value**: "GPT-4V(ision) is a Generalist Web Agent, if Grounded pdf"


In [ ]:
{"content": [{"type": "text", "text": prompt[1]}, 
            {"type": "image_url", "image_url": {"url": image_path,
                                                "detail": "high"},
                                                                }]}

In [12]:
await browser.perform_action(elements[8], "TYPE", "Logistic Regression")

Typed 'Logistic Regression' into element: aria-label="Search" name="q" title="Search"


'[textarea role="combobox"] aria-label="Search" name="q" title="Search" -> TYPE: Logistic Regression'

In [13]:
elements = await browser.get_current_page_elements()

In [16]:
await browser.perform_action(elements[8], "PRESS ENTER")

Pressed Enter on element: input value="Logistic Regression" aria-label="Search" name="q" title="Search"


'[textarea role="combobox"] input value="Logistic Regression" aria-label="Search" name="q" title="Search" -> PRESS ENTER'

In [17]:
elements = await browser.get_current_page_elements()

In [20]:
choices = format_choices(elements)
options = format_options(choices)

In [41]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4o",
    stop=["\nObservation", "Observation"]
)

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

In [31]:
await browser.stop()

Browser context closed.


In [44]:
await browser.take_screenshot()

In [48]:
elements[0]

{'center_point': (0.035, 0.042),
 'description': 'About',
 'tag_with_role': 'a',
 'box_raw': [21, 17, 46.59375, 26],
 'box': [21, 17, 67.59375, 43],
 'selector': <Locator frame=<Frame name= url='https://www.google.com/'> selector='a >> nth=0'>,
 'tag': 'a',
 'idx': 0,
 'option': 'A'}

In [47]:
elements[0]["selector"]

<Locator frame=<Frame name= url='https://www.google.com/'> selector='a >> nth=0'>

In [64]:
close_context = browser.session_control['context']
browser.session_control['context'] = None
await close_context.close()

In [12]:
browser.page

<Page url='https://www.google.com/'>

In [2]:
prompts = _initialize_prompts()

In [7]:
print("\n".join(wrap(prompts["system_prompt"])))

You are assisting humans doing web navigation tasks step by step. At
each stage, you can see the webpage by a screenshot and know the
previous actions before the current step decided by yourself that have
been executed for this task through recorded history. You need to
decide on the first following action to take.
